In [39]:
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open)
sleep(1)
homeCard = driver.find_elements(By.CSS_SELECTOR , "a.slider-item")
[c.get_attribute('href') for c in homeCard]

[]

In [26]:
import pandas as pd

In [27]:
from selenium import webdriver  # install selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep

In [28]:
path_to_browser = "./chromedriver.exe"
ser = Service(path_to_browser)
op = webdriver.ChromeOptions()
op.add_argument('headless')
page_link_to_open = "https://www.redfin.com/zipcode/00601"

In [111]:
# https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341
page_link_to_open2 = 'https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341'

driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open2)
sleep(0.2)
data = driver.find_elements(By.CSS_SELECTOR , ".statsValue")
[price, beds, bath, area] = [c.text for c in data]
stress_address = driver.find_elements(By.CSS_SELECTOR , ".street-address")[0].text
city_and_state = driver.find_elements(By.CSS_SELECTOR , ".dp-subtext")[0].text

# home_facts = driver.find_elements(By.CSS_SELECTOR, ".keyDetailsList")[0] \
#                    .find_elements(By.CSS_SELECTOR, "div.keyDetail")
# home_fact_index = [fact.find_elements(By.CSS_SELECTOR , "span")[0].text for fact in home_facts]
# home_fact_text = [fact.find_elements(By.CSS_SELECTOR, "span.text-right")[0].text for fact in home_facts]
# year_built = home_fact_text[home_fact_index.index('Year Built')] if 'Year Built' in home_fact_index else None
# lot_size = home_fact_text[home_fact_index.index('Lot Size')] if 'Lot Size' in home_fact_index else None

basic_infor = driver.find_elements(By.ID, "basicInfo")[0]
basic_infor_label = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "span.table-label")]
basic_infor_value = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "div.table-value")]
year_built = basic_infor_value[basic_infor_label.index('Year Built')] if 'Year Built' in basic_infor_label else None
county = basic_infor_value[basic_infor_label.index('County')] if 'County' in basic_infor_label else None
# lot_size = basic_infor_value[basic_infor_label.index('Lot Size')] if 'Lot Size' in basic_infor_label else None

scores = driver.find_elements(By.CSS_SELECTOR , ".walk-score")
if scores:
    scores = scores[0].find_elements(By.CSS_SELECTOR , ".score")
    trademark = [score.find_elements(By.CSS_SELECTOR , ".walkscore-trademark")[0].text[:-1] for score in scores]
    score_value = [score.find_elements(By.CSS_SELECTOR , "div.percentage > span.value")[0].text for score in scores]
    walk_score = score_value[trademark.index('Walk Score')] if 'Walk Score' in trademark else None
    transit_score = score_value[trademark.index('Transit Score')] if 'Transit Score' in trademark else None
    bike_score = score_value[trademark.index('Bike Score')] if 'Bike Score' in trademark else None

property_details_container = driver.find_elements(By.CSS_SELECTOR, "div.super-group-content > div.amenity-group")
heating_cooling_text = None
pool_text = None
parking_garage_text = None
lot_text = None

for pc in property_details_container:
    header = pc.find_elements(By.TAG_NAME, "h3")[0].text
    if header == 'Pool Information':
        pool_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Heating & Cooling':
        heating_cooling_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Parking & Garage Information':
        parking_garage_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Lot Information':
        lot_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]

photo_elements = driver.find_elements(By.CSS_SELECTOR, 'div.InlinePhotoPreview')
photo_elements = photo_elements[0] if photo_elements else []
img_href = " ".join([img.get_attribute('src') for img in photo_elements.find_elements(By.TAG_NAME, "img")])

In [112]:
img_href

'https://ssl.cdn-redfin.com/system_files/media/622779_JPG/item_10.jpg https://ssl.cdn-redfin.com/system_files/media/622779_JPG/genLdpUgcMediaBrowserUrl/item_29.jpg https://ssl.cdn-redfin.com/system_files/media/622779_JPG/genLdpUgcMediaBrowserUrl/item_85.jpg'

In [71]:
price, beds, bath, area, stress_address[:-1], city_and_state, year_built, lot_size, county, walk_score, transit_score, bike_score

('$7,900,000',
 '3',
 '3',
 '3,214',
 '401 Harrison St Unit 46B',
 'San Francisco, CA 94105',
 '2014',
 '—',
 'San Francisco County',
 '93',
 '100',
 '57')

In [84]:
for lt in lot_text:
    if lt.startswith('Lot Size Acres: '):
        lot_size = lt.split('Lot Size Acres: ')[1]

In [92]:
for hc in heating_cooling_text:
    if hc.startswith('Cooling: '):
        cooling = hc.split('Cooling: ')[1]
    if hc.startswith('Heating: '):
        heating = hc.split('Heating: ')[1]

In [98]:
for p in pool_text:
    if p.startswith('Has Private Pool: '):
        pooling = p.split('Has Private Pool: ')[1]

In [101]:
for pg in parking_garage_text:
    if pg.startswith('Parking Total: '):
        parking_size = pg.split('Parking Total: ')[1]

In [102]:
parking_size

'2'

In [111]:
import pandas as pd
import numpy as np
housing_price = pd.read_csv('data/housing_data_raw/0.csv', encoding='cp1252', error_bad_lines=False)
# price filter
housing_price = housing_price[(housing_price['price'].notna()) & (housing_price['price'].str.startswith('$'))]
housing_price['price'] = housing_price['price'].map(lambda x: x[1:]).map(lambda x: x if x[-1] != '+' else x[:-1])
housing_price.drop(columns=['lot_size.1', 'parking_size'], inplace=True)
# bath, bed, and areas required
housing_price = housing_price[(housing_price['bath'] != '—') & (housing_price['beds'] != '—') & (housing_price['area'].str.isnumeric())]
# Year column
mean_year = int(np.mean(housing_price[(housing_price['year_built'] != '—') & (housing_price['year_built'] != 'None')]['year_built'].astype(np.float)))
housing_price['year_built'] = housing_price['year_built'].map(lambda year: year if year != '—' and year != 'None' else mean_year).astype(np.int)
# Lot size
housing_price['lot_size'] = housing_price['lot_size'].map(lambda lot: lot if lot.isnumeric() else 0)
# score
housing_price['transit_score'] = housing_price['transit_score'].map(lambda score: score if score != 'None' else 0)
housing_price['walk_score'] = housing_price['walk_score'].map(lambda score: score if score != 'None' else 0)
housing_price['bike_score'] = housing_price['bike_score'].map(lambda score: score if score != 'None' else 0)
# cooling & heating
housing_price['cooling'] = housing_price['cooling'].map(lambda x: len(x.split('|')) if x != 'None' else 0)
housing_price['heating'] = housing_price['heating'].map(lambda x: len(x.split('|')) if x != 'None' else 0)
# has_pool
housing_price['has_pool'] = housing_price['has_pool'].map(lambda score: 1 if score == 'Yes' else 0)

b'Skipping line 235: expected 20 fields, saw 21\nSkipping line 252: expected 20 fields, saw 21\nSkipping line 721: expected 20 fields, saw 21\nSkipping line 1103: expected 20 fields, saw 21\nSkipping line 1451: expected 20 fields, saw 23\nSkipping line 1572: expected 20 fields, saw 21\nSkipping line 2357: expected 20 fields, saw 21\nSkipping line 2732: expected 20 fields, saw 22\nSkipping line 3193: expected 20 fields, saw 22\nSkipping line 3265: expected 20 fields, saw 23\nSkipping line 3586: expected 20 fields, saw 21\nSkipping line 3828: expected 20 fields, saw 21\nSkipping line 4727: expected 20 fields, saw 22\nSkipping line 5162: expected 20 fields, saw 22\nSkipping line 5367: expected 20 fields, saw 22\nSkipping line 5469: expected 20 fields, saw 21\nSkipping line 5535: expected 20 fields, saw 22\nSkipping line 5962: expected 20 fields, saw 21\nSkipping line 6149: expected 20 fields, saw 21\nSkipping line 6190: expected 20 fields, saw 21\nSkipping line 6635: expected 20 fields, s

In [112]:
housing_price

,href,stress_address,city,state_and_zip,county,img_href,beds,bath,area,year_built,lot_size,walk_score,transit_score,bike_score,cooling,heating,has_pool,price
0,https://www.redfin.com/TX/Dallas/3135-Royal-Ga...,3135 Royal Gable Dr,Dallas,TX 75229,Dallas County,https://ssl.cdn-redfin.com/photo/90/bigphoto/4...,3,2,1229,1983,0,68,39,63,0,0,0,274000
2,https://www.redfin.com/PA/Morrisville/42-E-Hen...,42 E Hendrickson Ave,Morrisville,PA 19067,Bucks County,https://ssl.cdn-redfin.com/photo/235/bigphoto/...,3,1,1348,1913,0,70,0,46,0,0,0,235000
5,https://www.redfin.com/VA/Richmond/4114-Bremne...,4114 Bremner Blvd,Henrico,VA 23228,Henrico County,https://ssl.cdn-redfin.com/photo/131/bigphoto/...,3,1,1434,1941,0,45,19,39,1,2,0,229000
6,https://www.redfin.com/MS/Jackson/1341-Deerfie...,1341 Deerfield Ln,Jackson,MS 39211,Hinds County,https://ssl.cdn-redfin.com/photo/294/bigphoto/...,3,2,1356,1961,0,11,15,33,2,2,0,139900
7,https://www.redfin.com/OH/Baltic/1390-Co-Rd-60...,1390 County Road 600,Baltic,OH 43804,Holmes County,https://ssl.cdn-redfin.com/photo/159/bigphoto/...,5,1.5,2266,1940,0,5,0,42,0,3,0,175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16204,https://www.redfin.com/OH/Barberton/1437-Saxon...,1437 Saxon Cir,Barberton,OH 44203,Summit County,https://ssl.cdn-redfin.com/photo/159/bigphoto/...,3,2.5,1870,1998,0,14,0,25,1,2,0,299900
16205,https://www.redfin.com/WA/Brush-Prairie/16205-...,16205 NE 189th Cir,Brush Prairie,WA 98606,Clark County,https://ssl.cdn-redfin.com/photo/84/bigphoto/7...,3,2,2500,1971,0,0,0,32,1,1,0,7900000
16210,https://www.redfin.com/MI/Detroit/2528-Helen-S...,2528 Helen St,Detroit,MI 48207,Wayne County,https://ssl.cdn-redfin.com/photo/144/bigphoto/...,6,4,2386,1910,0,33,38,68,0,0,0,165000
16211,https://www.redfin.com/DE/Wilmington/2103-N-Je...,2103 N Jefferson St,Wilmington,DE 19802,New Castle County,https://ssl.cdn-redfin.com/photo/235/bigphoto/...,4,2.5,2500,1912,0,74,50,69,0,0,0,217500


In [121]:
from collections import Counter
housing_price['price'].astype(np.float)

0         274000.0
2         235000.0
5         229000.0
6         139900.0
7         175000.0
           ...    
16204     299900.0
16205    7900000.0
16210     165000.0
16211     217500.0
16212    1299000.0
Name: price, Length: 10159, dtype: float64

In [78]:
len('Gas| Radiant| Wood'.split('|'))

3